In [2]:
################## preprocessing ###################
#################################################### 
####################################################
import pandas as pd
from os.path import isfile, join
import os
import logging
import argparse
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

date = '20220126'
basedir = '/home/lms/ms/poc'
rawdata_path = join(basedir, 'dataset')
procdata_path = join(basedir, 'deepfm', 'dataset', 'preprocess', 'rating')
procdata_path_r = join(basedir, 'deepfm', 'dataset', 'preprocess', 'rating_r')

In [3]:
user_data = pd.read_pickle(os.path.join(rawdata_path, 'user.pkl')) # unique id = 2047776
item_data = pd.read_pickle(os.path.join(rawdata_path, 'item.pkl'))
hist_data = pd.read_pickle(os.path.join(rawdata_path, 'history.pkl')) # unique id = 2038827

In [5]:
item_data.head(2)

,gubun,depth0_id,dpeth0_nm,genre,album_id,album_name,category_id,category_name,rep_album_id,rep_album_name,close_yn,series_yn,series_no,rank_no,run_time_ss,pr_info,price,release_date,point_watcha,summary
0,MOVIE,1,영화,영화,M01217E587PPV00,[우리말]루카(2021)(소장),M01217E587PPV00,[우리말]루카(2021)(소장),M01217E587PPV00,루카(,Y,Y,NaN,1.0,5740,1,6500,20210617.0,3.7,"바다 밖은 위험해?! 아니, 궁금해! 자칭 인간 세상 전문가인 친구와 스쿠터 여행을..."
1,MOVIE,1,영화,영화,M0121CM169PPV00,타이거 마스크(2021),M0121CM169PPV00,타이거 마스크(2021),M0121CM169PPV00,타이거,N,N,NaN,NaN,5830,3,10000,20211230.0,2.7,"존재감 없이 30여 년을 살아온 한 남자. 전설의 가디언, 타이거 마스크로 다시 태..."


In [4]:
mapping = pd.read_pickle(os.path.join(procdata_path, 'mapping.pkl'))
mapping

,sa_id,category_id,category_id_CODE,sa_id_CODE
0,205090929633,B3054,0,0
1,206059946955,B3054,0,1
2,207044266005,B3054,0,2
3,207044353341,B3054,0,3
4,208036007235,B3054,0,4
...,...,...,...,...
16044099,500228395772,M012141362PPV00,17216,1973456
16044100,500233024250,M012057404PPV00,17217,1973457
16044101,500079172794,M01208Q414PPV00,17218,1973458
16044102,500131191860,M01163M037PPV00,17219,1973459


In [5]:
items = pd.read_pickle(os.path.join(procdata_path, 'item.pkl'))
item_columns = ['category_id','gubun','rep_album_id','category_name']
category_data = pd.read_pickle(os.path.join(rawdata_path, 'item.pkl'))
category_data = category_data.astype({'category_id':'str', 'category_name':'str', 'album_id':'str'})
category_data = category_data.drop_duplicates(subset=item_columns)
category_data = items.merge(category_data, on='category_id')

In [6]:
map_category = category_data[['category_name', 'category_id_CODE', 'dpeth0_nm']]
map_category

,category_name,category_id_CODE,dpeth0_nm
0,꼬리에 꼬리를 무는 그날 이야기 시즌2,0,예능시사교양
1,표리부동,1,예능시사교양
2,쇼윈도：여왕의 집,2,드라마
3,워맨스가 필요해,3,예능시사교양
4,꼬리에 꼬리를 무는 그날 이야기 시즌1,4,예능시사교양
...,...,...,...
17216,에비에이터(소장),17216,영화
17217,딥워터 호라이즌,17217,영화
17218,위 캔 두 댓! 가치봄영화,17218,영화
17219,텍사스 레인저,17219,영화


In [7]:
mapping1 = pd.merge(mapping, map_category, how="left", on="category_id_CODE")
mapping1

,sa_id,category_id,category_id_CODE,sa_id_CODE,category_name,dpeth0_nm
0,205090929633,B3054,0,0,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
1,206059946955,B3054,0,1,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
2,207044266005,B3054,0,2,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
3,207044353341,B3054,0,3,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
4,208036007235,B3054,0,4,꼬리에 꼬리를 무는 그날 이야기 시즌2,예능시사교양
...,...,...,...,...,...,...
16044099,500228395772,M012141362PPV00,17216,1973456,에비에이터(소장),영화
16044100,500233024250,M012057404PPV00,17217,1973457,딥워터 호라이즌,영화
16044101,500079172794,M01208Q414PPV00,17218,1973458,위 캔 두 댓! 가치봄영화,영화
16044102,500131191860,M01163M037PPV00,17219,1973459,텍사스 레인저,영화


In [16]:
freq = mapping1['category_id_CODE'].value_counts()
freq1 = mapping1['category_id'].value_counts()

In [25]:
freq

94       290054
0        203378
59       191808
12       185488
32       147903
          ...  
12591         1
16650         1
14288         1
16649         1
16634         1
Name: category_id_CODE, Length: 17221, dtype: int64

In [23]:
map_category[map_category['category_id_CODE']==32]

,category_name,category_id_CODE,dpeth0_nm
32,그것이 알고 싶다,32,예능시사교양
